In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Tweet Analysis
2. Sentiment Analysis

**TWEET ANALYSIS**

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap
from plotly.subplots import make_subplots
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
trump = pd.read_csv('../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')

In [ ]:
trump.shape
trump.head()

In [ ]:
print('The different targets of trump in his tweets from the least cited to the most cited :')
print(trump.groupby(['target']).size().sort_values())


In [ ]:
print('The distribution of the 5 main targets of D.Trumps tweets :')
trump['target'].value_counts()[:5].sort_values(ascending=False).plot(kind="pie")

In [ ]:
print('The different insults used in his tweets :')
print(trump.groupby(['insult']).size().sort_values())

In [ ]:
print('The top 5 insults used in his tweets :')
trump['insult'].value_counts()[:5].sort_values(ascending=False).plot(kind="pie")


In [ ]:
trump_media = trump.loc[trump['target']=='hillary-clinton']
print('Most insult word with Hillary was :')

tweet_ALL = " ".join(insul for insul in trump_media.insult)

fig, ax = plt.subplots(1, 1, figsize = (12,10))

wordcloud_ALL = WordCloud(max_font_size=50, max_words=100,colormap="PuBu", background_color="white").generate(tweet_ALL)

ax.imshow(wordcloud_ALL, interpolation='bilinear')

ax.axis('off');

In [ ]:
trump_media = trump.loc[trump['target']=='democrats']
print('Most insult word against Democrats was :')


tweet_ALL = " ".join(insul for insul in trump_media.insult)

fig, ax = plt.subplots(1, 1, figsize = (12,10))

wordcloud_ALL = WordCloud(max_font_size=50, max_words=100,colormap="Blues", background_color="white").generate(tweet_ALL)

ax.imshow(wordcloud_ALL, interpolation='bilinear')

ax.axis('off');

In [ ]:
trump_media = trump.loc[trump['target']=='trump-russia']
print('Most insult word against Russia was :')


tweet_ALL = " ".join(insul for insul in trump_media.insult)

fig, ax = plt.subplots(1, 1, figsize = (12,10))

wordcloud_ALL = WordCloud(max_font_size=50, max_words=100,colormap="Reds", background_color="white").generate(tweet_ALL)

ax.imshow(wordcloud_ALL, interpolation='bilinear')

ax.axis('off');

In [ ]:
trump_media = trump.loc[trump['target']=='the-media']
print('Most insult word against The Media was :')

tweet_ALL = " ".join(insul for insul in trump_media.insult)

fig, ax = plt.subplots(1, 1, figsize = (12,10))

wordcloud_ALL = WordCloud(max_font_size=50, max_words=100,colormap="gist_heat", background_color="white").generate(tweet_ALL)

ax.imshow(wordcloud_ALL, interpolation='bilinear')

ax.axis('off');

![](https://zupimages.net/up/21/05/om1a.jpg)

In [ ]:
trump['date'] = pd.to_datetime(trump['date'])
trump['year'] = trump['date'].dt.year
trump['month'] = trump['date'].dt.month
trump['day'] =  trump['date'].dt.day
trump.head()

In [ ]:
print('The number of tweets posted with insults over the last few years :')
trump['date'].value_counts().resample('Y').sum()

sns.displot(trump['date'])

**SENTIMENT ANALYSIS**

In [ ]:
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import pandas as pd
import nltk
import textblob
import plotly.express as px
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.cluster import MiniBatchKMeans
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from nltk.sentiment.vader import SentimentIntensityAnalyzer as vad
import plotly
plotly.offline.init_notebook_mode (connected = True)

In [ ]:
data = trump

In [ ]:
# Proprocessing the data
data['tweet']=data['tweet'].str.lower()
# Code to remove the Hashtags from the text
data['tweet']=data['tweet'].apply(lambda x:re.sub(r'\B#\S+','',x))
# Code to remove the links from the text
data['tweet']=data['tweet'].apply(lambda x:re.sub(r"http\S+", "", x))
# Code to remove the Special characters from the text 
data['tweet']=data['tweet'].apply(lambda x:' '.join(re.findall(r'\w+', x)))
# Code to substitute the multiple spaces with single spaces
data['tweet']=data['tweet'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
# Code to remove all the single characters in the text
data['tweet']=data['tweet'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
# Remove the twitter handlers
data['tweet']=data['tweet'].apply(lambda x:re.sub('@[^\s]+','',x))

In [ ]:
data=data[['tweet']]


In [ ]:
# Invoking the TFIDFVectorizer
tf_data=TfidfVectorizer()
# Copying the data into a new dataframe called vader
vader=data.copy()

sentiment=vad()
# Making additional columns for sentiment score in the vader dataframe
sen=['Positive','Negative','Neutral']
sentiments=[sentiment.polarity_scores(i) for i in vader['tweet'].values]
vader['Negative Score']=[i['neg'] for i in sentiments]
vader['Positive Score']=[i['pos'] for i in sentiments]
vader['Neutral Score']=[i['neu'] for i in sentiments]
vader['Compound Score']=[i['compound'] for i in sentiments]
score=vader['Compound Score'].values
t=[]
for i in score:
    if i >=0.05 :
        t.append('Positive')
    elif i<=-0.05 :
        t.append('Negative')
    else:
        t.append('Neutral')
vader['Overall Sentiment']=t

In [ ]:
vader.head()

In [ ]:
fig=px.histogram(data_frame=vader,x='Compound Score',color='Overall Sentiment',template='plotly')
fig.show()

In [ ]:
sns.countplot(vader['Overall Sentiment'])

In [ ]:
vader_data=tf_data.fit_transform(vader['tweet'].values)


In [ ]:
pca = PCA(n_components=3).fit_transform(vader_data.todense())

In [ ]:
px.scatter_3d(x=pca[:,0],y=pca[:,1],z=pca[:,2],color=vader['Overall Sentiment'].values)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def clean_tweets(tweet):
 
    tweet = re.sub(r':', '', str(tweet))
    tweet = re.sub(r'‚Ä¶', '', str(tweet))
    tweet = re.sub('@[A-Za-z0–9]+', '', str(tweet))               
    tweet = re.sub('#', '', str(tweet)) 
    tweet = re.sub('https?:\/\/\S+', '', str(tweet)) 
    tweet = re.sub(r'[^\x00-\x7F]+',' ',str(tweet))
    tweet = str(tweet).lower()
    tweet = re.sub('\[.*?\]', '', tweet)
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub('<.*?>+', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\w*\d\w*', '', tweet)
    tweet = re.sub(r'#','',tweet)
    tweet = re.sub(r'RT[\s]+','',tweet)
    tweet = re.sub(r'[^\w]', ' ', tweet)
    tweet = re.sub('https', '', tweet)
    tweet = re.sub('https ', '', tweet)
    tweet = re.sub('co', '', tweet)

    return tweet

In [ ]:
def getSubjectivity(tweet):
    try:
        return TextBlob(tweet).sentiment.subjectivity
    except:
        return None

In [ ]:
def getPolarity(tweet):
    try:
        return  TextBlob(tweet).sentiment.polarity
    except:
        return None

In [ ]:
def sentiment_calc(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

In [ ]:
data['cleaned_text']=data['tweet'].apply(clean_tweets)

In [ ]:
data['subjectivity'] = data['cleaned_text'].apply(getSubjectivity)
data['polarity'] = data['cleaned_text'].apply(getPolarity)
data['sentiment'] = data['cleaned_text'].apply(sentiment_calc)

In [ ]:
def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

In [ ]:
data['analysis'] = data['polarity'].apply(getAnalysis)

In [ ]:
data.head()

In [ ]:
data.groupby('analysis')['analysis'].count().plot.bar()
plt.title('Sentiment Analyisis  ')

In [ ]:
x=data.groupby('analysis')['analysis'].count()
neutral=data[data['analysis']=='Neutral']['cleaned_text'].count()
positive=data[data['analysis']=='Positive']['cleaned_text'].count()
negative=data[data['analysis']=='Negative']['cleaned_text'].count()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=[go.Pie(labels=["positivity","negativity","neutrality"], values=[positive,negative,neutral])])
fig.update_layout(title_text='Trumps tweet sentiment analysis')
fig.show()